In [3]:
from mesa import Agent, Model
from mesa.space import NetworkGrid
from mesa.time import RandomActivation
import networkx as nx
import graph_init

In [ ]:
class Commuter(Agent):
    def __init__(self, model, distance_left, destination):
        super().__init__(model)
        self.distance_left = distance_left
        self.destination = destination

    def step(self):
        current_pos = self.position  # Access the agent's current position
        possible_moves = list(self.model.grid.get_neighbors(current_pos))
        if possible_moves:
            # random walk
            new_pos = self.model.random.choice(possible_moves)
            self.model.grid.move_agent(self, new_pos)
            print(f"Agent moved from node {current_pos} to node {new_pos}")
        else:
            print(f"Agent at node {current_pos} has no neighbors to move to")

class MyModel(Model):
    def __init__(self, n_agents):
        super().__init__()
        self.grid = NetworkGrid(graph_init.create_random_graph(10,5))
        self.schedule = RandomActivation(self)

        # Create agents and place them on the grid
        for i in range(1, n_agents + 1):
            node_id = self.random.choice(list(g.nodes()))
            a = Commute, se, ilf)
            self.schedule.add(a)
            self.grid.place_agent(a, node_id)
            print(f"Agent {a.unique_id} placed at node {node_id}")

    def step(self):
        print("Model step")
        self.schedule.step()